In [1]:
pip install tensorflow

  Using cached numpy-1.23.5-cp311-cp311-win_amd64.whl.metadata (2.3 kB)
Using cached numpy-1.23.5-cp311-cp311-win_amd64.whl (14.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
contourpy 1.3.3 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.23.5 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.23.5 which is incompatible.
scikit-learn 1.8.0 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.preprocessing import LabelEncoder
import pickle
import os
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [3]:
path = "train_data.csv"

df = pd.read_csv(path, low_memory=False)

In [4]:
path="test_data.csv"
df = pd.read_csv(path)

In [5]:
# ---------------- LOAD TRAIN & TEST CSV ----------------
train_df = pd.read_csv("train_data.csv", low_memory=False)
test_df  = pd.read_csv("test_data.csv", low_memory=False)

print("Train CSV:", train_df.shape)
print("Test CSV:", test_df.shape)

Train CSV: (30568, 64)
Test CSV: (5731, 64)


In [6]:
train_df.isnull().sum(axis=1)

0        0
1        0
2        0
3        0
4        0
        ..
30563    0
30564    0
30565    0
30566    0
30567    0
Length: 30568, dtype: int64

In [7]:
train_df[:1].dtypes

x0        object
y0        object
z0        object
x1        object
y1        object
           ...  
z19       object
x20       object
y20       object
z20       object
folder    object
Length: 64, dtype: object

In [8]:
train_df.head()

,x0,y0,z0,x1,y1,z1,x2,y2,z2,x3,...,x18,y18,z18,x19,y19,z19,x20,y20,z20,folder
0,0.449241579,0.64415729,-9.04e-08,0.429074943,0.635371089,0.003722572,0.408368379,0.63087821,0.002986991,0.392120898,...,0.391990364,0.655341327,-0.015527404,0.383398265,0.662171423,-0.015240635,0.37765485,0.669068992,-0.015004144,Whom|කාටද|யாருக்கு
1,0.382245362,0.558267474,7.06E-08,0.413395762,0.533815086,0.000568241,0.427210212,0.506034136,-0.001061654,0.434977144,...,0.402409852,0.479038984,-0.034060862,0.406095743,0.463534415,-0.036735639,0.409117639,0.449920237,-0.038406786,Ayubowan|ආයුබෝවන්|ஆயுபோவன்
2,0.452943265,0.621691525,-6.09E-08,0.468021393,0.626518607,0.001431449,0.479241788,0.636463523,0.001262602,0.485580653,...,0.449999154,0.665923119,-0.004571089,0.451644897,0.666010082,-0.001769416,0.451801866,0.662505746,0.000665507,Bag|බෑගය|பை
3,0.781940937,0.525786042,-7.77e-08,0.779762387,0.499152124,0.00450749,0.768690765,0.475780934,0.003003466,0.754124761,...,0.726171911,0.491400063,-0.025716092,0.730468273,0.498696953,-0.021514077,0.739267945,0.50162214,-0.017493377,I know|මම දන්නවා|எனக்கு தெரியும்
4,0.430419564,0.846907556,2.84e-08,0.470211208,0.821797192,-0.011500387,0.520999908,0.802827656,-0.025803007,0.56479913,...,0.503666461,0.881531835,-0.076967724,0.514606178,0.897112548,-0.074608542,0.51725775,0.908199072,-0.070945479,Tell|කියන්න|சொல்


In [9]:
X_train = train_df.drop("folder", axis=1)
y_train = train_df["folder"]

X_test = test_df.drop("folder", axis=1)
y_test = test_df["folder"]

In [10]:
X_train = X_train.apply(pd.to_numeric,errors="coerce")
X_test = X_test.apply(pd.to_numeric,errors="coerce")

In [11]:
X_test[:1].dtypes

x0     float64
y0     float64
z0     float64
x1     float64
y1     float64
        ...   
y19    float64
z19    float64
x20    float64
y20    float64
z20    float64
Length: 63, dtype: object

In [12]:
X_train[:1].dtypes

x0     float64
y0     float64
z0     float64
x1     float64
y1     float64
        ...   
y19    float64
z19    float64
x20    float64
y20    float64
z20    float64
Length: 63, dtype: object

In [13]:
X_train.isnull().sum(axis=1)

0        0
1        0
2        0
3        0
4        0
        ..
30563    0
30564    0
30565    0
30566    0
30567    0
Length: 30568, dtype: int64

In [14]:
np.isnan(X_train).sum()

x0     1
y0     1
z0     1
x1     1
y1     1
      ..
y19    1
z19    1
x20    1
y20    1
z20    1
Length: 63, dtype: int64

In [15]:
mask = X_train.notna().all(axis=1)

X_train = X_train.loc[mask]
y_train = y_train.loc[mask]

In [16]:
X_test[:5]

,x0,y0,z0,x1,y1,z1,x2,y2,z2,x3,...,z17,x18,y18,z18,x19,y19,z19,x20,y20,z20
0,0.441752,0.450380,1.790000e-07,0.461223,0.417520,0.001214,0.460240,0.383041,0.003527,0.456155,...,-0.010446,0.447635,0.374299,-0.015479,0.450875,0.360810,-0.016933,0.454070,0.349162,-0.017693
1,0.759249,0.559344,-1.060000e-07,0.721400,0.551557,-0.000053,0.682172,0.566878,-0.001861,0.658565,...,-0.021716,0.724178,0.640497,-0.022249,0.729266,0.632614,-0.015970,0.735672,0.623266,-0.011545
2,0.423867,0.686609,1.230000e-07,0.437681,0.666430,-0.005854,0.458200,0.662696,-0.008692,0.476785,...,-0.008212,0.480583,0.714617,-0.010361,0.492978,0.717287,-0.010956,0.502658,0.719305,-0.010956
3,0.358600,0.724326,-1.590000e-07,0.343545,0.709470,-0.003469,0.319427,0.695936,-0.007574,0.297728,...,-0.021964,0.298440,0.718010,-0.029165,0.285947,0.725180,-0.031624,0.276931,0.732808,-0.032822
4,0.578133,0.641648,4.540000e-07,0.557434,0.589512,0.007384,0.542337,0.546912,0.005396,0.529377,...,-0.031387,0.502241,0.552534,-0.034357,0.492158,0.528080,-0.034953,0.483088,0.508008,-0.036459


In [17]:
y_train[:5]

0                  Whom|කාටද|யாருக்கு
1          Ayubowan|ආයුබෝවන්|ஆயுபோவன்
2                         Bag|බෑගය|பை
3    I know|මම දන්නවා|எனக்கு தெரியும்
4                    Tell|කියන්න|சொல்
Name: folder, dtype: object

In [18]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

ENCODER_FILE = "label_encoder_382.pkl"
with open(ENCODER_FILE, "wb") as f:
    pickle.dump(label_encoder, f)

print(f"✅ Labels encoded and saved to '{ENCODER_FILE}'")

✅ Labels encoded and saved to 'label_encoder_382.pkl'


In [19]:
y_train_encoded[:5]

array([365,  12,  15, 158, 316])

In [20]:
len(label_encoder.classes_)

381

In [ ]:

# ---------------- BUILD MODEL ----------------
model = models.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.0),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.0),
    layers.Dense(32, activation='relu'),
    layers.Dense(len(label_encoder.classes_), activation='softmax')
])

# ---------- OPTIMIZER ----------
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# ---------------- CALLBACKS ----------------
checkpoint = ModelCheckpoint(
    filepath='Round_09/{epoch:02d}_accuracy_{val_accuracy:.4f}.keras',
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    mode='max'
)

early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=120,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.8,
    patience=30,
    min_lr=1e-6,
    verbose=1
)

# ---------------- TRAIN ----------------
history = model.fit(
    X_train, y_train_encoded,
    validation_split=0.1,
    epochs=450,
    batch_size=32,
    callbacks=[checkpoint, early_stop, reduce_lr],
    shuffle=True,
)


Epoch 1/450
855/860 [============================>.] - ETA: 0s - loss: 5.6120 - accuracy: 0.0106
Epoch 1: val_accuracy improved from -inf to 0.01603, saving model to Round_09\01_accuracy_0.0160.keras
860/860 [==============================] - 6s 5ms/step - loss: 5.6096 - accuracy: 0.0108 - val_loss: 5.3456 - val_accuracy: 0.0160 - lr: 0.0010
Epoch 2/450
848/860 [============================>.] - ETA: 0s - loss: 5.2213 - accuracy: 0.0249
Epoch 2: val_accuracy improved from 0.01603 to 0.02781, saving model to Round_09\02_accuracy_0.0278.keras
860/860 [==============================] - 4s 4ms/step - loss: 5.2190 - accuracy: 0.0251 - val_loss: 5.1278 - val_accuracy: 0.0278 - lr: 0.0010
Epoch 3/450
851/860 [============================>.] - ETA: 0s - loss: 4.8822 - accuracy: 0.0551
Epoch 3: val_accuracy improved from 0.02781 to 0.08374, saving model to Round_09\03_accuracy_0.0837.keras
860/860 [==============================] - 3s 3ms/step - loss: 4.8818 - accuracy: 0.0552 - val_loss: 4.674

In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   


 dense_24 (Dense)            (None, 128)               8192      
                                                                 
 dropout_12 (Dropout)        (None, 128)               0         
                                                                 
 dense_25 (Dense)            (None, 64)                8256      
                                                                 
 dropout_13 (Dropout)        (None, 64)                0         
                                                                 
 dense_26 (Dense)            (None, 32)                2080      
                                                                 
 dense_27 (Dense)            (None, 381)               12573     
                                                                 
Total params: 31,101
Trainable params: 31,101
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# ---------------- EVALUATE ----------------
loss, acc = model.evaluate(X_test, y_test_encoded, verbose=1)
print(f"\nTest Accuracy: {acc:.4f}")

180/180 [==============================] - 1s 3ms/step - loss: 3.3375 - accuracy: 0.8027

Test Accuracy: 0.8027


In [ ]:
# ---------- EVALUATE MODEL ----------
test_loss, test_acc = model.evaluate(X_test, y_test_encoded, verbose=1)

print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")


180/180 [==============================] - 0s 2ms/step - loss: 3.3375 - accuracy: 0.8027
Test Accuracy: 0.8027
Test Loss: 3.3375


In [ ]:
predicts = model.predict(X_test)

180/180 [==============================] - 1s 2ms/step


In [ ]:
X_test[:5]

array([[-0.31955782, -1.36518945,  0.96800579, -0.21289389, -1.51039442,
         0.7101578 , -0.23415137, -1.62275122,  1.0781816 , -0.2695505 ,
        -1.66240605,  1.21181543, -0.26121735, -1.70676598,  1.2559884 ,
        -0.36564914, -1.52750306,  1.34301595, -0.34586189, -1.53734653,
         1.15763052, -0.31450037, -1.61730443,  0.95559548, -0.28709409,
        -1.68483237,  0.78600577, -0.3788169 , -1.52275355,  1.22069059,
        -0.35318475, -1.59019862,  1.02244744, -0.30930329, -1.71320104,
         0.78056132, -0.27016179, -1.81582676,  0.59305184, -0.36057762,
        -1.51518914,  0.92056718, -0.33296876, -1.59473913,  0.72280413,
        -0.29333075, -1.70488747,  0.48061505, -0.26206038, -1.79176791,
         0.30052605, -0.31729205, -1.49873188,  0.56066364, -0.30775157,
        -1.55349132,  0.42572481, -0.28648949, -1.62877446,  0.26003058,
        -0.26302867, -1.68790864,  0.12533503],
       [ 1.6967636 , -0.37586805, -0.95474742,  1.50139498, -0.30540793,
   

In [ ]:
predicts[:1]

array([[0.00000000e+00, 3.51198487e-10, 4.62297348e-17, 0.00000000e+00,
        3.28895574e-22, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 3.23251564e-11, 0.00000000e+00, 0.00000000e+00,
        1.59207076e-26, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 7.11205175e-08, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.25486069e-12, 0.00000000e+00,
        1.35637428e-32, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.58082444e-21,
        0.00000000e+00, 6.37576060e-38, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.41838494e-11, 0.000000

In [ ]:
y_pred_classes = np.argmax(predicts, axis=1)

In [ ]:
y_pred_classes[:5]

array([163, 201, 361,  88,  63], dtype=int64)

In [ ]:
y_pred_decode = label_encoder.inverse_transform(y_pred_classes)

In [ ]:
test_df["pred"]= y_pred_decode

In [ ]:
test_df.to_csv("new_output_qwe.csv", index=False)

In [ ]:
savedModel = tf.keras.models.load_model("model.keras")

ValueError: File not found: filepath=model.keras. Please ensure the file is an accessible `.keras` zip file.

In [ ]:
savedModel.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 128)               8192      
                                                                 
 dropout_10 (Dropout)        (None, 128)               0         
                                                                 
 dense_21 (Dense)            (None, 64)                8256      
                                                                 
 dropout_11 (Dropout)        (None, 64)                0         
                                                                 
 dense_22 (Dense)            (None, 32)                2080      
                                                                 
 dense_23 (Dense)            (None, 381)               12573     
                                                                 
Total params: 31,101
Trainable params: 31,101
Non-trai

In [ ]:
preds = savedModel.predict(X_test)

180/180 [==============================] - 1s 3ms/step


In [ ]:
preds[:5]

array([[0.0000000e+00, 1.1952403e-12, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [3.8146254e-37, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 6.5574077e-06, 2.2856662e-19, ..., 1.2008936e-09,
        2.0190686e-37, 2.4348173e-10],
       [1.9500741e-35, 1.0455498e-26, 1.0108445e-27, ..., 0.0000000e+00,
        0.0000000e+00, 1.1445886e-21],
       [0.0000000e+00, 2.1988718e-09, 7.8058445e-30, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]], dtype=float32)

In [ ]:
y_pred_Num = np.argmax(preds, axis=1)

In [ ]:
y_pred_Num[:5]

array([277, 201, 145,  88, 177], dtype=int64)